In [ ]:
from scipy.spatial.transform import Rotation as R
import h5py


def create_data(input_hdf5_path):
    f = h5py.File(input_hdf5_path)
    increments = list(range(0, 121, 10))

    output_data = {}

    for inc in increments:
        increment = f'increment_{inc}'
        output_data[increment] = {}
        phase_data = f[f'increment_{inc}']['phase']['Aluminum']['mechanical']
        geometry_data = f[f'increment_{inc}']['geometry']

        for key in phase_data.keys():
            output_data[increment][key] = phase_data[key][:]
        
        for key in geometry_data.keys():
            output_data[increment][key] = geometry_data[key][:]
    
    return output_data

def convert_quart_to_euler(orientations):
    """
    Input: orientations, [x,y,z,w] quarternion
    """
    euler_orientations = R.from_quat(orientations).as_euler('XYZ') # 'XYZ' for intrinsic rotation
    
    return euler_orientations

In [ ]:
def match_file_path(type, direction):
    return fr'2D_10%strain_100grains\results_{type}{direction}\crystal_100_50x1x50_{type}{direction}_material.hdf5'

types = ['Tension', 'Compression']
directions = ['X', 'XZ', 'Z']

all_output_data = {}

for type in types:
    for direction in directions:
        output_data = create_data(match_file_path(type, direction))
        for increment in output_data.keys():
            # print(output_data[increment]['O'])
            euler_orientations = convert_quart_to_euler(output_data[increment]['O'])
            output_data[increment]['O_euler'] = euler_orientations

            all_output_data[f'{type}{direction}'] = output_data


dict_keys(['TensionX', 'TensionXZ', 'TensionZ', 'CompressionX', 'CompressionXZ', 'CompressionZ'])


In [13]:
print(all_output_data.keys())
print(all_output_data['TensionX']['increment_0']['O'][100], all_output_data['TensionX']['increment_0']['O_euler'][100])

dict_keys(['TensionX', 'TensionXZ', 'TensionZ', 'CompressionX', 'CompressionXZ', 'CompressionZ'])
[ 0.11500406 -0.24943454  0.10530306 -0.95575506] [-0.19453619  0.52477375 -0.16706881]
